In [1]:
import rasterio
import geopandas as gpd
import pyproj
import os
import matplotlib.pyplot as plt

In [2]:
def snow_depth(path, DEM, outdir):
    
    for file in os.listdir(path):
        if file.endswith('shp'):
            filename = file[0:4]
            points = os.path.join(path,file)

            pts = gpd.read_file(points)

            pts['lon'] = pts.geometry.x
            pts['lat'] = pts.geometry.y

            coords = [(x,y) for x, y in zip(pts.lon, pts.lat)]

            src = rasterio.open(DEM)

            pts['DEM'] = [x[0] for x in src.sample(coords)]

            pts['SD'] = round(pts.Est_Surf - pts.DEM,2)

            pts = pts[pts['SD'] < 10]
            pts = pts[pts['SD'] > -10]

            pts.to_file(outdir+'SD_'+filename+'.shp')
            print('processing completed for beam: ', filename)
        

In [3]:
DEM = 'C:/Users/Rasmu/Documents/Thesis/DEM/mergedDEM10m.tif'
path = 'C:/Users/Rasmu/Documents/Thesis/Surface_Estimation_2/'
outdir = 'C:/Users/Rasmu/Documents/Thesis/Snow_Depth_2/'

In [4]:
snow_depth(path, DEM, outdir)

processing completed for beam:  gt1l
processing completed for beam:  gt1r
processing completed for beam:  gt2l
processing completed for beam:  gt2r
processing completed for beam:  gt3l
processing completed for beam:  gt3r
